# Estimation of Obesity Levels Based On Eating Habits and Physical Conditions
Source: https://archive.ics.uci.edu/dataset/544/estimation+of+obesity+levels+based+on+eating+habits+and+physical+condition \
Key Features: gender, age, height, weight, eating habits, physical conditon, lifestyle habits

In [14]:
### Verify all libraries are installed in your machine first. 

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 

# Other libraries that may be useful in the future. 
# import duckdb # for using sql syntax to access a df 
# import pandasql # also for using sql to access df - have slightly different features /advantages"
# import pil # python imaging library for opening, manipulating and saving images

## Functions that may be useful. 
## %%sql

In [11]:
uci_df = pd.read_csv("DataSets/UCI_ObesityDataSet.csv")
uci_df.head()


,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking,Overweight_Level_I
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II


In [12]:
uci_cell_count = uci_df.size
uci_row_count = uci_df.shape[0]
uci_column_count = uci_df.shape[1]

print(uci_cell_count)
print(uci_row_count)
print(uci_column_count)

35887
2111
17


## Analysis